In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#load movies list
movieList = pd.read_csv('movies2016.csv')

In [3]:
movieList.head()

,name,id,year,director,top,rating,num_of_rates
0,Дэдпул,tt1431045,2016,"Tim Miller (dir.), Ryan Reynolds, Morena Baccarin",NaN,8.0,679175
1,Бэтмен против Супермена: На заре справедливости,tt2975590,2016,"Zack Snyder (dir.), Ben Affleck, Henry Cavill",NaN,6.6,504695
2,Первый мститель: Противостояние,tt3498820,2016,"Anthony Russo (dir.), Chris Evans, Robert Down...",NaN,7.9,451069
3,Отряд самоубийц,tt1386697,2016,"David Ayer (dir.), Will Smith, Jared Leto",NaN,6.2,445485
4,Прибытие,tt2543164,2016,"Denis Villeneuve (dir.), Amy Adams, Jeremy Renner",NaN,8.0,406478


## Get reviews by movie id:

In [4]:
def ParseMovie(movie_id):
    movieReviews = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 'numOfReviews',
                               'review_name', 'author', 'region', 'date', 'review_text'])
    
    
    #at first let's get info about the movie by id
    url = "http://www.imdb.com/title/" + movie_id + "/reviews"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    #get movie_name
    tn15title = soup.findChild(id="tn15title")
    movie_name = tn15title.find_all("a")[0].get_text()
    tn15content = soup.find(id="tn15content")
    numOfReviews = int(tn15content.find_all("table")[2].get_text().replace("\n"," ").split(" ")[3])
    print movie_name, numOfReviews, "---------------"
    #download reviews
    for k in xrange(numOfReviews/10+1):
        #create url
        start_review = 10*k
        url = "http://www.imdb.com/title/" + str(movie_id) + "/reviews?start=" + str(start_review)
       # print  (1.0*start_review / numOfReviews)
        #download html
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        tn15content = soup.find(id="tn15content")

        #delete message about spoilers
        tn15_p = list(tn15content.find_all("p"))
        len1 = len(tn15_p)
        i=0
        while(i<len1):
            if tn15_p[i].get_text() == u'*** This review may contain spoilers ***':
                tn15_p.remove(tn15_p[i])
                i =-1
                len1 -= 1
            i+=1
        tn15_p.remove(tn15_p[len1-1])
        len1 -= 1 #number of review on the current page

        #preparate data for all the reviews on the page
        tn15_review = tn15content.find_all(class_='yn')

        tn15_small = tn15content.find_all("small")#useful, date, region
        review_name = tn15content.find_all("h2")
        tn15_div = tn15content.find_all("div")

        for i in xrange(len1):
            review = {}
            review['review_id'] = tn15_review[i].attrs['id'].split("_")[1]
            review['review_text'] = tn15_p[i].get_text().replace("\n", " ").lower()
            review['movie_name'] = movie_name
            review['movie_id'] = movie_id
            review['numOfReviews'] = numOfReviews
            review['review_name'] = review_name[i].get_text()
            review['author'] = tn15_div[2*i+0].find_all("a")[1].get_text()
            review['author_id'] = tn15_div[2*i+0].find_all("a")[1].attrs['href'].split('/')[2]

            #check if we have rating
            if (len(tn15content.find_all('div')[2*i].find_all('img'))) == 1:
                review['rating'] = None
            else: 
                review['rating'] = tn15content.find_all("div")[2*i].find_all("img")[1].attrs['alt'].split("/")[0]



            #check if we have region and uselful
            tn15_small = tn15_div[2*i].find_all('small')

            if len(tn15_small) == 3:
                review['useful'] = int(tn15_small[0].get_text().split(' ')[0])
                review['useful_of'] = int(tn15_small[0].get_text().split(" ")[3])
                review['region'] = tn15_small[1].get_text()[5:]
                review['date'] = tn15_small[2].get_text()

            elif len(tn15_small) == 1:
                review['date'] = tn15_small[0].get_text()
                review['useful'] = None
                review['useful_of'] = None
                review['region'] = None

            elif len(tn15_small) == 2:
                review['date'] = tn15_small[1].get_text()
                if tn15_small[0].get_text()[0] == u'f':
                    review['region'] = tn15_small[0].get_text()[5:]
                    review['useful'] = None
                    review['useful_of'] = None
                else:
                    review['region'] = None
                    review['useful'] = int(tn15_small[0].get_text().split(' ')[0])
                    review['useful_of'] = int(tn15_small[0].get_text().split(' ')[3])

            movieReviews = movieReviews.append([review], ignore_index=True)
    return movieReviews


# Save reviews by movies id:

In [5]:
imdb = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 'numOfReviews',
                               'review_name', 'author', 'author_id', 'region', 'date', 'review_text'])

In [ ]:
#get reviews from 0 to 99
for i in xrange(0,1500):
    print i
    imdb = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 'numOfReviews',
                               'review_name', 'author', 'author_id', 'region', 'date', 'review_text'])
    
    try:
        
        imdb = imdb.append(ParseMovie(movieList.id[i]))
        
        cvs_name = './2016/reviews_' + str(i) +'.csv'
        imdb.to_csv(cvs_name,encoding='utf-8', index=False)
    except:
        print u'фейл((('

# Put all reviews in one csv:

In [27]:
imdb = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 
                               'numOfReviews', 'review_name', 'author', 'region', 'date', 'review_text'])

#5, 23, 182
for i in xrange(0,1500):
    try:
        cvs_name = './2016/reviews_' + str(i) + '.csv'
        print cvs_name
        imdb1 = pd.read_csv(cvs_name)
        imdb = imdb.append(imdb1, ignore_index=True)
    except:
        print i, 'fail'
imdb.to_csv('imbd.csv',encoding='utf-8', index=False)

./2016/reviews_0.csv
./2016/reviews_1.csv
./2016/reviews_2.csv
./2016/reviews_3.csv
./2016/reviews_4.csv
./2016/reviews_5.csv
./2016/reviews_6.csv
./2016/reviews_7.csv
./2016/reviews_8.csv
./2016/reviews_9.csv
./2016/reviews_10.csv
./2016/reviews_11.csv
./2016/reviews_12.csv
./2016/reviews_13.csv
./2016/reviews_14.csv
./2016/reviews_15.csv
./2016/reviews_16.csv
./2016/reviews_17.csv
./2016/reviews_18.csv
./2016/reviews_19.csv
./2016/reviews_20.csv
./2016/reviews_21.csv
./2016/reviews_22.csv
./2016/reviews_23.csv
./2016/reviews_24.csv
./2016/reviews_25.csv
./2016/reviews_26.csv
./2016/reviews_27.csv
./2016/reviews_28.csv
./2016/reviews_29.csv
29 fail
./2016/reviews_30.csv
./2016/reviews_31.csv
./2016/reviews_32.csv
32 fail
./2016/reviews_33.csv
./2016/reviews_34.csv
./2016/reviews_35.csv
./2016/reviews_36.csv
36 fail
./2016/reviews_37.csv
./2016/reviews_38.csv
./2016/reviews_39.csv
./2016/reviews_40.csv
./2016/reviews_41.csv
./2016/reviews_42.csv
./2016/reviews_43.csv
./2016/reviews_44.

./2016/reviews_360.csv
./2016/reviews_361.csv
./2016/reviews_362.csv
./2016/reviews_363.csv
./2016/reviews_364.csv
./2016/reviews_365.csv
./2016/reviews_366.csv
./2016/reviews_367.csv
./2016/reviews_368.csv
./2016/reviews_369.csv
./2016/reviews_370.csv
./2016/reviews_371.csv
./2016/reviews_372.csv
./2016/reviews_373.csv
373 fail
./2016/reviews_374.csv
374 fail
./2016/reviews_375.csv
./2016/reviews_376.csv
376 fail
./2016/reviews_377.csv
./2016/reviews_378.csv
./2016/reviews_379.csv
./2016/reviews_380.csv
./2016/reviews_381.csv
./2016/reviews_382.csv
./2016/reviews_383.csv
./2016/reviews_384.csv
./2016/reviews_385.csv
./2016/reviews_386.csv
./2016/reviews_387.csv
./2016/reviews_388.csv
./2016/reviews_389.csv
./2016/reviews_390.csv
./2016/reviews_391.csv
./2016/reviews_392.csv
./2016/reviews_393.csv
./2016/reviews_394.csv
./2016/reviews_395.csv
./2016/reviews_396.csv
./2016/reviews_397.csv
./2016/reviews_398.csv
./2016/reviews_399.csv
./2016/reviews_400.csv
./2016/reviews_401.csv
./2016/

./2016/reviews_706.csv
706 fail
./2016/reviews_707.csv
./2016/reviews_708.csv
./2016/reviews_709.csv
./2016/reviews_710.csv
./2016/reviews_711.csv
./2016/reviews_712.csv
./2016/reviews_713.csv
./2016/reviews_714.csv
./2016/reviews_715.csv
./2016/reviews_716.csv
./2016/reviews_717.csv
./2016/reviews_718.csv
./2016/reviews_719.csv
./2016/reviews_720.csv
./2016/reviews_721.csv
./2016/reviews_722.csv
./2016/reviews_723.csv
723 fail
./2016/reviews_724.csv
./2016/reviews_725.csv
725 fail
./2016/reviews_726.csv
./2016/reviews_727.csv
./2016/reviews_728.csv
./2016/reviews_729.csv
./2016/reviews_730.csv
./2016/reviews_731.csv
./2016/reviews_732.csv
./2016/reviews_733.csv
./2016/reviews_734.csv
./2016/reviews_735.csv
./2016/reviews_736.csv
./2016/reviews_737.csv
./2016/reviews_738.csv
./2016/reviews_739.csv
./2016/reviews_740.csv
./2016/reviews_741.csv
./2016/reviews_742.csv
./2016/reviews_743.csv
./2016/reviews_744.csv
./2016/reviews_745.csv
./2016/reviews_746.csv
./2016/reviews_747.csv
./2016/

./2016/reviews_1038.csv
./2016/reviews_1039.csv
./2016/reviews_1040.csv
./2016/reviews_1041.csv
./2016/reviews_1042.csv
./2016/reviews_1043.csv
./2016/reviews_1044.csv
./2016/reviews_1045.csv
./2016/reviews_1046.csv
./2016/reviews_1047.csv
./2016/reviews_1048.csv
./2016/reviews_1049.csv
./2016/reviews_1050.csv
./2016/reviews_1051.csv
./2016/reviews_1052.csv
./2016/reviews_1053.csv
./2016/reviews_1054.csv
./2016/reviews_1055.csv
./2016/reviews_1056.csv
1056 fail
./2016/reviews_1057.csv
./2016/reviews_1058.csv
1058 fail
./2016/reviews_1059.csv
./2016/reviews_1060.csv
1060 fail
./2016/reviews_1061.csv
./2016/reviews_1062.csv
./2016/reviews_1063.csv
./2016/reviews_1064.csv
./2016/reviews_1065.csv
./2016/reviews_1066.csv
./2016/reviews_1067.csv
./2016/reviews_1068.csv
./2016/reviews_1069.csv
1069 fail
./2016/reviews_1070.csv
./2016/reviews_1071.csv
./2016/reviews_1072.csv
1072 fail
./2016/reviews_1073.csv
./2016/reviews_1074.csv
./2016/reviews_1075.csv
./2016/reviews_1076.csv
./2016/reviews

./2016/reviews_1349.csv
./2016/reviews_1350.csv
./2016/reviews_1351.csv
1351 fail
./2016/reviews_1352.csv
1352 fail
./2016/reviews_1353.csv
./2016/reviews_1354.csv
./2016/reviews_1355.csv
./2016/reviews_1356.csv
./2016/reviews_1357.csv
./2016/reviews_1358.csv
1358 fail
./2016/reviews_1359.csv
./2016/reviews_1360.csv
./2016/reviews_1361.csv
./2016/reviews_1362.csv
./2016/reviews_1363.csv
./2016/reviews_1364.csv
1364 fail
./2016/reviews_1365.csv
./2016/reviews_1366.csv
./2016/reviews_1367.csv
1367 fail
./2016/reviews_1368.csv
1368 fail
./2016/reviews_1369.csv
./2016/reviews_1370.csv
./2016/reviews_1371.csv
1371 fail
./2016/reviews_1372.csv
./2016/reviews_1373.csv
1373 fail
./2016/reviews_1374.csv
./2016/reviews_1375.csv
./2016/reviews_1376.csv
./2016/reviews_1377.csv
1377 fail
./2016/reviews_1378.csv
1378 fail
./2016/reviews_1379.csv
./2016/reviews_1380.csv
./2016/reviews_1381.csv
./2016/reviews_1382.csv
./2016/reviews_1383.csv
./2016/reviews_1384.csv
./2016/reviews_1385.csv
./2016/revie

In [28]:
df = pd.read_csv('imdb_2016.csv')

In [29]:
df

,author,author_id,date,movie_id,movie_name,numOfReviews,rating,region,review_id,review_name,review_text,useful,useful_of
0,gogoschka-1,ur15794099,12 February 2016,tt1431045,Deadpool,1227,8.0,wherever good films play,3413536,"Dirty, funny, sexy, violent and unabashedly ro...","i really, really liked deadpool. and you know...",719.0,1113.0
1,moviewizguy,ur8334868,15 February 2016,tt1431045,Deadpool,1227,8.0,United States,3415632,This is what it looks like when filmmakers tak...,deadpool is a triumph of artistic vision over...,539.0,927.0
2,jelimafish,ur53058848,17 February 2016,tt1431045,Deadpool,1227,10.0,NaN,3416994,Best Movie I Have Seen In A Long Time,firstly i would like to state that it is comp...,463.0,884.0
3,GoodHandsomeLad,ur50186340,13 February 2016,tt1431045,Deadpool,1227,10.0,United States,3414233,Simply Excellent.,deadpool is a 2016 film that the eighth film ...,332.0,630.0
4,peterafro,ur66428645,26 September 2016,tt1431045,Deadpool,1227,10.0,United States,3551439,a much needed breath of fresh air in a stale g...,"at first glance, deadpool seems like a typica...",53.0,92.0
5,Asif Khan (asifahsankhan),ur55828906,14 December 2016,tt1431045,Deadpool,1227,9.0,"London, England",3598955,"""Bad Deadpool Good Deadpool Film!""",if anyone else could be cast to play deadpool...,31.0,52.0
6,Time Saver,ur68949569,18 October 2016,tt1431045,Deadpool,1227,8.0,NaN,3564194,"Funny, entertaining and refreshing",i am aware that to many people (especially th...,31.0,53.0
7,William Harrison,ur76123862,10 June 2017,tt1431045,Deadpool,1227,10.0,NaN,3727595,A new landmark for superhero and comedy films,"with a brilliant script, visceral action, and...",16.0,25.0
8,emma munns,ur34317310,24 May 2016,tt1431045,Deadpool,1227,8.0,United Kingdom,3475401,"Well, blow me!",let me say at the very beginning of this revi...,47.0,87.0
9,Ineffable,ur77126597,7 June 2017,tt1431045,Deadpool,1227,10.0,USA,3725810,Defend the 10 again,i've said it in other reviews that no movie i...,13.0,20.0


In [30]:
df.rating.value_counts()

10.0    63336
9.0     30740
8.0     25498
1.0     24558
7.0     19500
6.0     16061
5.0     14935
3.0     12117
4.0     11106
2.0     10945
Name: rating, dtype: int64